<a href="https://colab.research.google.com/github/Lee-Gunju/AI-paper-code-review-for-personal-project/blob/master/node_classification_withGAT_on_spektral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install spektral

     |████████████████████████████████| 122kB 26.9MB/s 


In [ ]:
"""
This example implements the experiments on citation networks from the paper:
Graph Attention Networks (https://arxiv.org/abs/1710.10903)
Petar Veli
"""

import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout, Input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.random import set_seed

from spektral.data.loaders import SingleLoader
from spektral.datasets.citation import Citation
from spektral.layers import GATConv
from spektral.transforms import AdjToSpTensor, LayerPreprocess


In [ ]:
dataset = Citation(
    "cora", normalize_x=True, transforms=[LayerPreprocess(GATConv), AdjToSpTensor()]
)

Pre-processing node features


/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
dataset.n_nodes

2708

In [ ]:
def mask_to_weights(mask):
    return mask.astype(np.float32) / np.count_nonzero(mask)

In [ ]:
weights_tr, weights_va, weights_te = (
    mask_to_weights(mask)
    for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te)
)

In [ ]:
# Parameters
channels = 8  # Number of channels in each head of the first GAT layer
n_attn_heads = 8  # Number of attention heads in first GAT layer
dropout = 0.6  # Dropout rate for the features and adjacency matrix
l2_reg = 2.5e-4  # L2 regularization rate
learning_rate = 5e-3  # Learning rate
epochs = 20000  # Number of training epochs
patience = 100  # Patience for early stopping


In [ ]:
N = dataset.n_nodes  # Number of nodes in the graph
F = dataset.n_node_features  # Original size of node features
n_out = dataset.n_labels  # Number of classes


In [ ]:
# Model definition
x_in = Input(shape=(F,))
a_in = Input((N,), sparse=True)

In [ ]:
graph = dataset.graphs

In [ ]:
graph[0].x.shape

(2708, 1433)

In [ ]:
graph[0].y.shape

(2708, 7)

In [ ]:
dataset.n_node_features

1433

In [ ]:
do_1 = Dropout(dropout)(x_in)
gc_1 = GATConv(
    channels,
    attn_heads=n_attn_heads,
    concat_heads=True,
    dropout_rate=dropout,
    activation="elu",
    kernel_regularizer=l2(l2_reg),
    attn_kernel_regularizer=l2(l2_reg),
    bias_regularizer=l2(l2_reg),
)([do_1, a_in])
do_2 = Dropout(dropout)(gc_1)
gc_2 = GATConv(
    n_out,
    attn_heads=1,
    concat_heads=False,
    dropout_rate=dropout,
    activation="softmax",
    kernel_regularizer=l2(l2_reg),
    attn_kernel_regularizer=l2(l2_reg),
    bias_regularizer=l2(l2_reg),
)([do_2, a_in])


In [ ]:
# Build model
model = Model(inputs=[x_in, a_in], outputs=gc_2)
optimizer = Adam(lr=learning_rate)
model.compile(
    optimizer=optimizer,
    loss=CategoricalCrossentropy(reduction="sum"),
    weighted_metrics=["acc"],
)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1433)]       0                                            
__________________________________________________________________________________________________
dropout (Dropout)               (None, 1433)         0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2708)]       0                                            
__________________________________________________________________________________________________
gat_conv (GATConv)              (None, 64)           91904       dropout[0][0]                    
                                                                 input_2[0][0]              

In [ ]:
# Train model
loader_tr = SingleLoader(dataset, sample_weights=weights_tr)
loader_va = SingleLoader(dataset, sample_weights=weights_va)

In [ ]:
loader_tr.load()

<RepeatDataset shapes: (((2708, 1433), (2708, 2708)), (2708, 7), (2708,)), types: ((tf.float32, tf.float32), tf.float32, tf.float32)>

In [ ]:
model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
)

Epoch 1/20000
1/1 [==============================] - 5s 5s/step - loss: 1.9501 - acc: 0.0786 - val_loss: 1.9492 - val_acc: 0.2080
Epoch 2/20000
1/1 [==============================] - 0s 53ms/step - loss: 1.9469 - acc: 0.2571 - val_loss: 1.9465 - val_acc: 0.5080
Epoch 3/20000
1/1 [==============================] - 0s 51ms/step - loss: 1.9445 - acc: 0.4571 - val_loss: 1.9447 - val_acc: 0.3780
Epoch 4/20000
1/1 [==============================] - 0s 50ms/step - loss: 1.9414 - acc: 0.3857 - val_loss: 1.9439 - val_acc: 0.4760
Epoch 5/20000
1/1 [==============================] - 0s 51ms/step - loss: 1.9407 - acc: 0.4286 - val_loss: 1.9430 - val_acc: 0.4260
Epoch 6/20000
1/1 [==============================] - 0s 49ms/step - loss: 1.9383 - acc: 0.4214 - val_loss: 1.9417 - val_acc: 0.4500
Epoch 7/20000
1/1 [==============================] - 0s 47ms/step - loss: 1.9367 - acc: 0.4643 - val_loss: 1.9402 - val_acc: 0.4940
Epoch 8/20000
1/1 [==============================] - 0s 53ms/step - loss: 1.93

In [ ]:
# Evaluate model
print("Evaluating model.")
loader_te = SingleLoader(dataset, sample_weights=weights_te)
eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Evaluating model.
1/1 [==============================] - 0s 28ms/step - loss: 0.9751 - acc: 0.8320
Done.
Test loss: 0.9750693440437317
Test accuracy: 0.8320000171661377
